<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/QAS_BERT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import json
from platform import python_version

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,MultiHeadAttention

import tensorflow_hub as hub
!pip install -q tensorflow-text

import tensorflow_text as text

In [2]:
print('Python Version : ' ,python_version())
print('tensorflow version : ', tf.__version__)
print('pandas version : ', pd.__version__)
print('numpy version : ', np.__version__)
print('tf_hub version : ', hub.__version__)


Python Version :  3.7.10
tensorflow version :  2.4.1
pandas version :  1.1.5
numpy version :  1.19.5
tf_hub version :  0.11.0


In [3]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
# max_seq_length = 384

In [4]:
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['title'][0:])
print(raw_train_data['data'][1]['title'][0:])

dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
University_of_Notre_Dame
Beyoncé


In [5]:
print(raw_train_data['data'][3]['paragraphs'][0].keys())

dict_keys(['context', 'qas'])


In [6]:
print(raw_train_data['data'][0]['paragraphs'][0]['qas'])

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}, {'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}, {'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}, {'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}]


In [7]:
print(raw_train_data['data'][0]['title'])
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['paragraphs'][0]['qas'][1]['answers'][0]['text'])
x = (raw_train_data['data'][0]['paragraphs'][0]['context'])
x

University_of_Notre_Dame
dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
a copper statue of Christ


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [8]:
raw_train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']

515

In [9]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

bert_preprocess_layer = hub.load(bert_preprocess)
bert_layer = hub.KerasLayer(bert_model,trainable=True)

In [10]:
def fun(x):
  return (bert_preprocess_layer.tokenize([str(x)]))

In [11]:
den = ['hi']
fun(den)

<tf.RaggedTensor [[[1031], [1005], [7632], [1005], [1033]]]>

In [12]:
def text_maker (data):
  squad_examples = []
  full_context = []
  full_question = []
  full_answer = []
  title = []
  starting_index = []
  context_question =[]
  new_line =[]
  ending_index = []

  for item in data["data"]:
   
      title.append(item['title'])
      for para in item["paragraphs"]:
          context = para["context"]
          # full_context.append(context)

          

          for qa in para["qas"]:
              question = qa["question"]
              x = np.hstack((context,question))
              new_line.append(x)

              

              if "answers" in qa:
                  answer_text = qa["answers"][0]["text"]
                  full_answer.append(answer_text)
                  all_answers = [_["text"] for _ in qa["answers"]]
                  start_char_idx = qa["answers"][0]["answer_start"]
                  end_char_idx = start_char_idx + len(answer_text)
                  # print(start_char_idx)
                  starting_index.append(start_char_idx)
                  ending_index.append(end_char_idx)
                # squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
            # else:
                # squad_eg = Sample(question, context)
            # squad_eg.preprocess()
            # squad_examples.append(squad_eg)
  x  = pd.DataFrame(new_line)
  # x.rename(columns ={0:"context", 1:"questions"})
  x['start_index'] = starting_index
  x['end_index'] = ending_index
  
  return (x)

In [13]:
col1 = ['I live in london','weather is nice']
col2 = ['I am old.', 'my name is john']

In [14]:
train_data= (text_maker(raw_train_data)).rename(columns ={0:"context", 1:"questions"})
test_data = (text_maker(raw_eval_data)).rename(columns ={0:"context", 1:"questions"})
# all_data[0].head()

In [15]:
slicer = []
for i in range(len(train_data)):
  slicer.append('SLI')

train_data['slicer']=slicer

In [16]:
train_data.head(3)

,context,questions,start_index,end_index,slicer
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,541,SLI
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,213,SLI
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,296,SLI


In [17]:
test_data.head(3)

,context,questions,start_index,end_index
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,191
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,266
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,403,426


In [18]:
def bert_tokenizer(x):
  # a = x.numpy()
  return (bert_preprocess_layer.tokenize(x))

def bert_deneme(x):
  return (print(x))

In [19]:
flag = ['i live in london']

bert_tokenizer(flag)

<tf.RaggedTensor [[[1045], [2444], [1999], [2414]]]>

In [29]:
train_dataset_context=tf.data.Dataset.from_tensor_slices(train_data['context'])
train_dataset_context =train_dataset_context.batch(30)
train_dataset_ques = tf.data.Dataset.from_tensor_slices(train_data['questions'])
train_dataset_ques = train_dataset_ques.batch(30)

In [30]:
train_question_tokens = []
for i in train_dataset_ques:
  train_question_tokens.append(bert_tokenizer(i))

In [31]:
train_context_tokens = []
for i in train_dataset_context:
  train_context_tokens.append(bert_tokenizer(i))

In [36]:
len(train_context_tokens[0].numpy())

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/ragged/ragged_tensor.py:2012: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(rows)


30

In [34]:
train_question_tokens[0]

<tf.RaggedTensor [[[2000], [3183], [2106], [1996], [6261], [2984], [9382], [3711], [1999], [8517], [1999], [10223, 26371], [2605], [1029]], [[2054], [2003], [1999], [2392], [1997], [1996], [10289], [8214], [2364], [2311], [1029]], [[1996], [13546], [1997], [1996], [6730], [2540], [2012], [10289], [8214], [2003], [3875], [2000], [2029], [3252], [1029]], [[2054], [2003], [1996], [24665, 23052], [2012], [10289], [8214], [1029]], [[2054], [7719], [2006], [2327], [1997], [1996], [2364], [2311], [2012], [10289], [8214], [1029]], [[2043], [2106], [1996], [24105], [2932], [1997], [10289], [8214], [4088], [4640], [1029]], [[2129], [2411], [2003], [10289], [8214], [1005], [1055], [1996], [26536, 17420], [2405], [1029]], [[2054], [2003], [1996], [3679], [3076], [3259], [2012], [10289], [8214], [2170], [1029]], [[2129], [2116], [3076], [2739], [4981], [2024], [2179], [2012], [10289], [8214], [1029]], [[1999], [2054], [2095], [2106], [1996], [3076], [3259], [2691], [3168], [4088], [4772], [2012], [

In [38]:
x = bert_preprocess_layer.bert_pack_inputs([train_context_tokens[0] , train_question_tokens[0]])

In [41]:
x['input_word_ids'][0]

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  6549,  2135,  1010,  1996,  2082,  2038,  1037,  3234,
        2839,  1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,
        8514,  2003,  1037,  3585,  6231,  1997,  1996,  6261,  2984,
        1012,  3202,  1999,  2392,  1997,  1996,  2364,  2311,  1998,
        5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,  4828,
        2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,
        2310,  3490,  2618,  4748,  2033, 18168,  5267,  1000,  1012,
        2279,  2000,  1996,  2364,  2311,  2003,  1996, 13546,  1997,
        1996,  6730,  2540,  1012,  3202,  2369,  1996, 13546,  2003,
        1996, 24665, 23052,  1010,  1037, 14042,  2173,  1997,  7083,
        1998,  9185,  1012,  2009,  2003,  1037, 15059,  1997,  1996,
       24665, 23052,  2012, 10223, 26371,  1010,  2605,  2073,  1996,
        6261,  2984, 22353,   102,  2000,  3183,  2106,  1996,  6261,
        2984,  9382,  3711,  1999,  8517,  1

In [ ]:
def bert_tokenizer(x):
  return (bert_preprocess_layer.tokenize([x]))

def joiner(x,y):
  x_tokens = bert_tokenizer(x)
  y_tokens = bert_tokenizer(y)
  return (bert_preprocess_layer.bert_pack_inputs([x_tokens,y_tokens]))

In [ ]:
x = bert_preprocess_layer.tokenize(['nice weather'])

In [ ]:
# input_layer = keras.Input(shape = ())
# bert_layer_nn1 , bert_layer_nn2,bert_layer_nn3,bert_layer_nn4  = bert_layer(a_and_b)
# end_layer = layers.Dense(1)(tf.constant(bert_layer_nn1))
# model_nn = keras.Model(inputs=input_layer, output = end_layer)
# loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# optimizer = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
# model.compile(optimizer=optimizer, loss=[loss, loss])
# model.summary()
# model.fit([a,b], y_deneme, epochs=2, batch_size=8)
